# **Trabalho Individual 2**

Nome: João Pedro Assumpção Evaristo

RA: 147887
 
Para a aplicação do algoritmo genético no problema das 8 rainhas, optei por tratar cada indivíduo dentro do código como um vetor com as 8 posições, representando as 8 colunas em que as rainhas poderiam estar.
 
Inicialmente uma população é gerada a partir de uma função que escolhe uma posição aleatória em um intervalo de 0 a 8 para cada espaço no vetor. Após isso, é separado uma porcentagem de indivíduos que possuem um fitness melhor, baseado na porcentagem de elitismo. Após isso, com base na taxa de cruzamento, aqueles indivíduos com melhor fitness são selecionados para um cruzamento. Durante essa etapa, é selecionado um ponto de corte aleatório no qual será feito o crossover entre os indivíduos. Pode acontecer uma mutação nessa etapa dependendo da taxa de mutação aplicada. Esses filhos são adicionados à população daquele geração. Posteriormente, serão selecionados os melhores indivíduos da população, que irão fazer parte da próxima geração. Assim, o algoritmo segue, até que o número de gerações seja alcançado, ou caso um indivíduo com um fitness de 28 (fitness é calculado como 28 - número de ataques no tabuleiro) seja encontrado.


In [ ]:
from collections import Counter
from random import randint, random, choices
import numpy as np
from numpy.random import choice


def fazer_combinacao(ind1, ind2, corte):
    filho1 = ind1[:corte] + ind2[corte:]
    filho2 = ind2[:corte] + ind1[corte:]
    return [filho1, filho2]


def gerar_populacao(quantidade):
    p_inicial = []
    for individuo in range(quantidade):
        p_inicial.append([randint(1, 8) for i in range(8)])
    return p_inicial


def decisao(probabilidade):
    return random() < probabilidade


def cruzamento(ind1, ind2, taxa_mutacao):
    ponto_corte = randint(0, 7) # Ponto de corte aleatorio
    filho1 = ind1[:ponto_corte] + ind2[ponto_corte:]
    filho2 = ind2[:ponto_corte] + ind1[ponto_corte:]
    if decisao(taxa_mutacao): # Mutacao ocorre de acordo com a porcentagem de chance
        mutacao(filho1)
        mutacao(filho2)
    return [filho1, filho2]


def mutacao(individuo):
    gene_mutado = randint(0, 7)
    mut = randint(1, 8)
    while individuo[gene_mutado] == mut:  # evita do gene continuar sendo o mesmo apos a multacao
        mut = randint(1, 8)
    individuo[gene_mutado] = mut


def seleciona_populacao(populacao, quantidade):
    fitness_total = 0
    for p in populacao:
        fit = fitness(p)
        if fit == 28:  # Caso o fitness seja 0, significa que nao ha nenhum ataque no tabuleiro
            return [p]
        fitness_total += fit
    populacao_aux = sorted(populacao, key=lambda x: fitness(x), reverse=True)
    escolhidos = populacao_aux[:quantidade]  # Seleciona aqueles que tiveram melhores fitness
    return escolhidos


def fitness(individuo):
    score = 28 - contagem_ataques(individuo) # 28 e o numero maximo de ataques que podem ocorrer em um tabuleiro
    return score


def algoritmo_genetico(populacao, geracao, elitismo, t_cruzamento, t_mutacao):
    g = 0
    achou = False
    P = gerar_populacao(populacao)
    while g != geracao or achou:
        populacao_ordenada = sorted(P, key=lambda x: fitness(x), reverse=True)
        quant_elite = elitismo * len(P)
        populacao_ordenada = populacao_ordenada[int(quant_elite):]  # aplica o elitismo separando os melhores
        pais = choice([i for i in range(len(populacao_ordenada))], int(t_cruzamento * len(P)), replace=False)
        pais = [populacao_ordenada[i] for i in pais]
        if len(pais) % 2 == 0:  # Numero de pais par:
            for i in range(0, len(pais), 2):
                P.extend(cruzamento(pais[i], pais[i + 1], t_mutacao))
        else:  # Numero de pais impar
            for i in range(0, (len(pais) - 1), 2):
                P.extend((cruzamento(pais[i], pais[i + 1], t_mutacao)))
            P.extend(cruzamento(pais[0], pais[-1], t_mutacao))  # cruzamento do primeiro com o ultimo
        P = seleciona_populacao(P, populacao)  # P passa a ser os melhores
        if len(P) == 1:
            achou = True
            print(f'Tabuleiro vencedor {P[0]}')
            break
        print(f'Geracao {g}: {[contagem_ataques(i) for i in P]}')
        g += 1


def contagem_ataques(individuo):
    pesos = [7, 5, 3, 1, -1, -3, -5, -7]
    ataques = 0
    diagonais_1 = [x + individuo[x] for x in range(8)] # diagonal para cima
    diagonais_2 = [x + individuo[x] + pesos[x] for x in range(8)] # diagonal para baixo
    qtd = 0
    for i in range(8):
        qtd += Counter(individuo)[individuo[i]] - 1 # linha reta
        qtd += Counter(diagonais_1)[diagonais_1[i]] - 1
        qtd += Counter(diagonais_2)[diagonais_2[i]] - 1
    ataques += int(qtd / 2)
    return ataques


if __name__ == "__main__":
      algoritmo_genetico(30, 10, 0.1, 0.5, 0.1)


Geracao 0: [4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9]
Geracao 1: [1, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8]
Geracao 2: [1, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
Geracao 3: [1, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7]
Geracao 4: [1, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
Geracao 5: [1, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6]
Geracao 6: [1, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Geracao 7: [1, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Geracao 8: [1, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Geracao 9: [1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4


</head>

<body>
<table cellspacing="0" border="0">
	<colgroup span="4" width="85"></colgroup>
	<tr>
		<td height="92" align="left"><b><font face="Liberation Serif">Configuração </font></b></td>
		<td align="left"><font face="Liberation Serif">%Sucessos </font></td>
		<td align="left"><font face="Liberation Serif">%Falhas </font></td>
		<td align="left"><font face="Liberation Serif">#Geração (Média, Mínima e Máxima) para Sucesso e Falha</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 5, 0.1, 0.3, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 5, 0.2, 0.5, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 5, 0.2, 0.3, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 5, 0.1, 0.5, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 5, 0.1, 0.3, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 5, 0.2, 0.5, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 5, 0.2, 0.3, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 5, 0.1, 0.5, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 5, 0.1, 0.3, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 5, 0.2, 0.3, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 5, 0.1, 0.5, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 5, 0.1, 0.3, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 5, 0.2, 0.5, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 5, 0.2, 0.3, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 5, 0.2, 0.5, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 5, 0.1, 0.5, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="5" sdnum="1033;"><font face="Liberation Serif">5</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 10, 0.1, 0.3, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 10, 0.2, 0.3, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 10, 0.1, 0.5, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 10, 0.1, 0.3, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 10, 0.2, 0.5, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 10, 0.2, 0.5, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 10, 0.1, 0.5, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">10, 10, 0.2, 0.3, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 10, 0.1, 0.3, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 10, 0.2, 0.3, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 10, 0.1, 0.5, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 10, 0.1, 0.3, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 10, 0.2, 0.5, 0.1</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 10, 0.2, 0.3, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 10, 0.2, 0.5, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
	<tr>
		<td height="32" align="left"><font face="Liberation Serif">30, 10, 0.1, 0.5, 0.5</font></td>
		<td align="right" sdval="0" sdnum="1033;"><font face="Liberation Serif">0</font></td>
		<td align="right" sdval="100" sdnum="1033;"><font face="Liberation Serif">100</font></td>
		<td align="right" sdval="10" sdnum="1033;"><font face="Liberation Serif">10</font></td>
	</tr>
</table>
<!-- ************************************************************************** -->
</body>

</html>


## **Conclusão**
 
A partir da tabela do relatório, é possível notar que o algoritmo genético, nessas condições, não foi capaz de encontrar uma solução. Por outro lado, acompanhando as rodadas é possível notar que os tabuleiros se aproximam de números baixos de ataque, entre 1 e 3. Fazendo alguns testes, vi que é necessário um número maior de gerações para atingir um resultado, cerca de 20 no mínimo, para começar a observar tabuleiros vencedores.